<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>

# 颜色跟随  Color Following

In [1]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Red': ("Red", "红色"),
    'Green': ("Green", "绿色"),
    'Blue': ("Blue", "蓝色"),
    'Yellow': ("Yellow", "黄色"),
    'Start': ("Start", "开始"),
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [2]:
# 显示摄像头组件 Camera display widgets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2
import time
import numpy as np
import threading

In [3]:
#bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [4]:
from SunriseRobotLib import Mipi_Camera

img_width=640
img_height=480

g_camera = Mipi_Camera(img_width, img_height)

if g_camera.isOpened():
    print("Open Camera OK")
else:
    print("Fail To Open Camera")


Camera 0:
	i2c_bus: 6
	mipi_host: 0
Camera 1:
	i2c_bus: 4
	mipi_host: 2
Camera 2:
	i2c_bus: 0
	mipi_host: 0
Camera 3:
	i2c_bus: 0
	mipi_host: 0
mipi mclk is not configed.
Searching camera sensor on device: /proc/device-tree/soc/cam/vcon@0 i2c bus: 6 mipi rx phy: 0
INFO: Found sensor name:imx219-30fps on mipi rx csi 0, i2c addr 0x10, config_file:linear_1920x1080_raw10_30fps_2lane.c
2000/01/01 09:02:57.008 !INFO [CamInitParam][0279]Setting VSE channel-0: input_width:1920, input_height:1080, dst_w:640, dst_h:480
Open Camera OK
2000/01/01 09:02:57.008 !INFO [CamInitParam][0279]Setting VSE channel-1: input_width:1920, input_height:1080, dst_w:1920, dst_h:1080
2000/01/01 09:02:57.008 !INFO [vp_vin_init][0041]csi0 ignore mclk ex attr, because mclk is not configed at device tree.


In [5]:
#Color_HSV = {
#    'Red': ([0, 43, 46], [10, 255, 255]),
#    'Green': ([35, 43, 46], [77, 255, 255]),
#    'Blue': ([100, 43, 46], [124, 255, 255]),
#    'Yellow': ([26, 43, 46], [34, 255, 255]),
#    'Custom': ([0, 0, 0], [180, 255, 255]),
#}
Color_HSV  = {"Red"   : ([0, 111, 67], [16, 255, 255]),
              "Green" : ([43, 90, 65], [90, 255, 255]),
              "Blue"  : ([100, 43, 46], [124, 255, 255]),
              "Yellow": ([17, 138, 106], [34, 255, 255]),
              "Custom": ([0, 0, 0], [180, 255, 255])}

# 默认选择红色的，
# The default is red.
color_lower = np.array(Color_HSV["Red"][0])
color_upper = np.array(Color_HSV["Red"][1])


g_stop_program = False
g_start_function = False

color_x = color_y = color_radius = 0

In [6]:
# image_widget = widgets.Image(format='jpeg', width=320, height=240)
# 摄像头显示控件  Camera display widgets
DISPLAY_WIDTH = img_width
DISPLAY_HEIGHT = img_height
image_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)

In [7]:
# 颜色选择按钮配置  Color selection button configuration
# 红色 Red
Button_Red = widgets.Button(       
    description=Name_widgets['Red'][g_ENABLE_CHINESE],
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' )

# 绿色 Green
Button_Green = widgets.Button(     
    description=Name_widgets['Green'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 蓝色 Blue
Button_Blue = widgets.Button(        
    description=Name_widgets['Blue'][g_ENABLE_CHINESE],         
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 黄色 Yellow
Button_Yellow = widgets.Button(        
    description=Name_widgets['Yellow'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 开始 Start
Button_Start = widgets.Button(        
    description=Name_widgets['Start'][g_ENABLE_CHINESE],        
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 关闭摄像头 Close_Camera
Button_Close_Camera = widgets.Button(
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

In [8]:
# 清除按键显示状态  Clear button display status
def ALL_Uncheck():
    Button_Red.icon = 'uncheck'
    Button_Green.icon = 'uncheck'
    Button_Blue.icon = 'uncheck'
    Button_Yellow.icon = 'uncheck'


# 按键按下事件处理   Key press event processing
def on_button_clicked(b):
    global color_lower, color_upper, g_stop_program
    ALL_Uncheck()
    b.icon = 'check'
    with output:
        print("Button clicked:", b.description)
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        g_stop_program = True
        time.sleep(.1)
        g_camera.release()
        b.icon = 'uncheck'
        Button_Start.icon = 'uncheck'
    elif b.description == Name_widgets['Red'][g_ENABLE_CHINESE]:
        color_lower = np.array(np.array(Color_HSV["Red"][0]))
        color_upper = np.array(np.array(Color_HSV["Red"][1]))
    elif b.description == Name_widgets['Green'][g_ENABLE_CHINESE]:
        color_lower = np.array(np.array(Color_HSV["Green"][0]))
        color_upper = np.array(np.array(Color_HSV["Green"][1]))
    elif b.description == Name_widgets['Blue'][g_ENABLE_CHINESE]:
        color_lower = np.array(np.array(Color_HSV["Blue"][0]))
        color_upper = np.array(np.array(Color_HSV["Blue"][1]))
    elif b.description == Name_widgets['Yellow'][g_ENABLE_CHINESE]:
        color_lower = np.array(np.array(Color_HSV["Yellow"][0]))
        color_upper = np.array(np.array(Color_HSV["Yellow"][1]))


# 功能按钮按下事件处理 Function button press event handling
def on_button_function_clicked(b):
    global g_start_function
    if b.description == Name_widgets['Start'][g_ENABLE_CHINESE]:
        if b.icon == 'uncheck':
            b.icon = 'check'
            g_start_function = True
        else:
            b.icon = 'uncheck'
            g_start_function = False
            time.sleep(.1)
            g_car.set_car_motion(0, 0, 0)
    with output:
        print("Button clicked:", b.description, g_start_function)

In [9]:
# 关联按键事件回调 Button event callbacks
Button_Red.on_click(on_button_clicked)
Button_Green.on_click(on_button_clicked)
Button_Blue.on_click(on_button_clicked)
Button_Yellow.on_click(on_button_clicked)
Button_Close_Camera.on_click(on_button_clicked)
Button_Start.on_click(on_button_function_clicked)

In [10]:
# import PID
from simplePID import PID

PID_X_Kp = 0.02
PID_X_Ki = 0
PID_X_Kd = 0.005

PID_Y_Kp = 0.02
PID_Y_Ki = 0
PID_Y_Kd = 0.005

PID_X = PID(960, PID_X_Kp, PID_X_Ki, PID_X_Kd)
PID_Y = PID(540, PID_Y_Kp, PID_Y_Ki, PID_Y_Kd)


In [11]:
from SunriseRobotLib import SunriseRobot
g_car = SunriseRobot()

def car_control(x, y, radius):
    global last_x, last_y, last_radius
    speed_x = PID_X.incremental(y)/300.0
    speed_y = PID_Y.incremental(x)/300.0
    speed_z = 0
    '''
    if -0.1 < speed_x < 0.1:
        speed_x = 0
    if -0.1 < speed_y < 0.1:
        speed_y = 0
    if speed_x > 0.5:
        speed_x = 0.5
    if speed_x < -0.5:
        speed_x = -0.5
    if speed_y > 0.5:
        speed_y = 0.5
    if speed_y < -0.5:
        speed_y = -0.5
    print("speed:", speed_x, speed_y, speed_z)
    print("x:", x, "y:", y)
    '''
    g_car.set_car_motion(speed_x, speed_y, speed_z)
    last_x, last_y, last_radius = x, y, radius


Sunrise Robot Serial Opened! Baudrate=115200


In [12]:
# 图像处理任务 Image processing tasks
def task_processing():
    global color_lower, color_upper
    global g_stop_program, g_start_function
    t_start = time.time()
    m_fps = 0
    fps = 0
    while g_camera.isOpened():
        if g_stop_program:
            break
        ret, frame = g_camera.read()
        
        # 根据HSV值处理图像 The image is processed according to the HSV value
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, color_lower, color_upper)

        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2] 
        if len(cnts) > 0:
            cnt = max (cnts, key = cv2.contourArea)
            (color_x, color_y), color_radius = cv2.minEnclosingCircle(cnt)
            if color_radius > 10:
                # 将检测到的颜色用圆形线圈标记出来 Mark the detected colors with circular coils
                cv2.circle(frame, (int(color_x), int(color_y)), int(1), (0,255,0), 2)
                cv2.circle(frame, (int(color_x), int(color_y)), int(color_radius), (255,0,255), 2)
            if g_start_function:
                car_control(int(color_x), int(color_y), int(color_radius))
                # time.sleep(.01)
        
        m_fps = m_fps + 1
        fps = m_fps / (time.time() - t_start)
        if (time.time() - t_start) >= 2:
            m_fps = fps
            t_start = time.time() - 1        
        if g_start_function:
            cv2.putText(frame, "START " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        else:
            cv2.putText(frame, "FPS " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        
        # 图像传输给显示组件 The image is transmitted to the display component
        image_widget.value = bgr8_to_jpeg(frame)


In [13]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=task_processing)
thread1.daemon = True
thread1.start()

# display(image_widget)
# 创建一个横向的盒子容器，以便将图像小部件相邻放置
# create a horizontal box container to place the image widget next to eachother
image_container = widgets.HBox([image_widget])
button_group = widgets.HBox([Button_Red, Button_Green, Button_Blue, Button_Yellow, Button_Start, Button_Close_Camera])
output = widgets.Output()
box_display = widgets.VBox([image_container, button_group, output])
display(box_display)